##### Identify PCI

In [ ]:
# Identify PCI skills via fuzzy matching
import pandas as pd
import numpy as np
from difflib import SequenceMatcher
from tqdm import tqdm

df1 = pd.read_parquet(r'import1.parquet')
df2 = pd.read_parquet(r'import2.parquet')

keywords = df2['PCI_keywords'].dropna().unique()

# Fuzzy matching function
def fuzzy_match(skill_mapped):
    if pd.isna(skill_mapped):
        return np.nan, 0.0
    scores = [(kw, SequenceMatcher(None, str(skill_mapped).lower(), str(kw).lower()).ratio()) 
              for kw in keywords]
    best_kw, best_score = max(scores, key=lambda x: x[1])
    return best_kw, best_score

tqdm.pandas(desc="Fuzzy Matching Progress")

df1[['PCI_keywords', 'skills_match_score']] = df1['skill_mapped'].progress_apply(
    lambda x: pd.Series(fuzzy_match(x))
)

print(f"\nMatching completed! Statistics:")
print(f"Max match score: {df1['skills_match_score'].max():.4f}")
print(f"Min match score: {df1['skills_match_score'].min():.4f}")
print(f"Average match score: {df1['skills_match_score'].mean():.4f}")
print(f"\nTop 5 matching results:")
print(df1[['skill_mapped', 'PCI_keywords', 'skills_match_score']].head())

df1.to_parquet(r'output.parquet', index=False)

In [ ]:
# Identify PCI skills
import pandas as pd

df1 = pd.read_parquet(
    "imput.parquet",
    columns=['user_id', 'skills_match_score', 'is_ai_broad']
)

# Add PCI skill flag column
df1['is_pci_skill'] = (df1['skills_match_score'] >= 0.7).astype(int)

# Drop unused column
df1 = df1.drop(columns=['skills_match_score'])

df1 = df1.drop_duplicates(ignore_index=True)

df1.to_parquet("output.parquet", index=False)

In [ ]:
# Identify PCI employees
import pandas as pd

df1 = pd.read_parquet("imput.parquet")

df1['is_pci'] = (df1['pci_score'] >= 0.3).astype(int)

df1 = df1[['user_id', 'is_pci', 'is_ai_broad']]
df1 = df1.drop_duplicates()

df1.to_parquet("output.parquet", index=False)

##### Identify AI

In [ ]:
# Identify AI employees
import pandas as pd
import numpy as np
from difflib import SequenceMatcher
from tqdm import tqdm

df1 = pd.read_parquet("imput1.parquet")
df2 = pd.read_excel("imput2.xlsx")

keywords = df2['AI_keywords'].dropna().unique()

# Fuzzy matching function
def fuzzy_match(skill_mapped):
    if pd.isna(skill_mapped):
        return np.nan, 0.0
    scores = [(kw, SequenceMatcher(None, str(skill_mapped).lower(), str(kw).lower()).ratio()) 
              for kw in keywords]
    best_kw, best_score = max(scores, key=lambda x: x[1])
    return best_kw, best_score

tqdm.pandas(desc="Fuzzy Matching Progress")

df1[['AI_keywords', 'ai_skills_match_score']] = df1['skill_mapped'].progress_apply(
    lambda x: pd.Series(fuzzy_match(x))
)

print(f"\nMatching completed! Statistics:")
print(f"Max match score: {df1['ai_skills_match_score'].max():.4f}")
print(f"Min match score: {df1['ai_skills_match_score'].min():.4f}")
print(f"Average match score: {df1['ai_skills_match_score'].mean():.4f}")
print(f"\nTop 5 matching results:")
print(df1[['skill_mapped', 'AI_keywords', 'ai_skills_match_score']].head())

df1.to_parquet(r'output.parquet', index=False)

In [ ]:
# Identify AI employees
import pandas as pd

df1 = pd.read_parquet(
    "imput.parquet",
    columns=['user_id', 'ai_skills_match_score']
)

df1['is_ai_broad'] = (df1['ai_skills_match_score'] >= 0.75).astype(int)

df1 = df1.drop(columns=['ai_skills_match_score'])

df1 = df1.drop_duplicates(ignore_index=True)

df1.to_parquet("output.parquet", index=False)

##### Identify AIPCI

In [ ]:
# Identify AIPCI employees
import pandas as pd

df1 = pd.read_parquet("imput.parquet")

df1['is_ai_pci'] = ((df1['is_pci'] == 1) & (df1['is_ai_broad'] == 1)).astype(int)

target_columns = ['is_pci', 'is_ai_broad', 'is_ai_pci']
for col in target_columns:
    print(f"=== {col}")
    unique_user_count = df1.groupby(col)['user_id'].nunique()
    print(unique_user_count)
    print()
    
df1.to_parquet(r'output.parquet', index=False)

##### Calculate AI/PCI/AIPCI share

In [ ]:
# Calculate AI/PCI/AIPCI workforce share
import pandas as pd

df1 = pd.read_parquet("imput.parquet")[
    ['rcid', 'year_quarter', 'PCI_workers_it', 'AI_workers_it', 'AIPCI_workers_it', 'total_workforce_it']
]

# Calculate share columns (threshold = 2)
df1['AI_workforce_share_it_2'] = df1.apply(
    lambda x: x['AI_workers_it'] / x['total_workforce_it'] if x['total_workforce_it'] >= 2 else 0, axis=1
)
df1['PCI_workforce_share_it_2'] = df1.apply(
    lambda x: x['PCI_workers_it'] / x['total_workforce_it'] if x['total_workforce_it'] >= 2 else 0, axis=1
)
df1['AIPCI_workforce_share_it_2'] = df1.apply(
    lambda x: x['AIPCI_workers_it'] / x['total_workforce_it'] if x['total_workforce_it'] >= 2 else 0, axis=1
)

# Calculate share columns (threshold = 5)
df1['AI_workforce_share_it_5'] = df1.apply(
    lambda x: x['AI_workers_it'] / x['total_workforce_it'] if x['total_workforce_it'] >= 5 else 0, axis=1
)
df1['PCI_workforce_share_it_5'] = df1.apply(
    lambda x: x['PCI_workers_it'] / x['total_workforce_it'] if x['total_workforce_it'] >= 5 else 0, axis=1
)
df1['AIPCI_workforce_share_it_5'] = df1.apply(
    lambda x: x['AIPCI_workers_it'] / x['total_workforce_it'] if x['total_workforce_it'] >= 5 else 0, axis=1
)

df1.to_parquet("output.parquet", index=False)

##### Set initial stock $C_{i,0}$

In [ ]:
import pandas as pd

df = pd.read_parquet("imput.parquet")

# Define variable pairs for initial stock calculation
init_var_pairs = [
    ('AI_workforce_share_it_2', 'AIC_init_stock_2'),
    ('PCI_workforce_share_it_2', 'PCIC_init_stock_2'),
    ('AIPCI_workforce_share_it_2', 'AIPCI_init_stock_2'),
    ('AI_workforce_share_it_5', 'AIC_init_stock_5'),
    ('PCI_workforce_share_it_5', 'PCIC_init_stock_5'),
    ('AIPCI_workforce_share_it_5', 'AIPCI_init_stock_5')
]

# Calculate and merge initial stock for each firm
for centered_stock_var, init_stock_var in init_var_pairs:
    def get_first_non_null(group):
        non_null_vals = group[centered_stock_var].dropna()
        if not non_null_vals.empty:
            return non_null_vals.iloc[0]
        else:
            return group[centered_stock_var].mean()
    
    init_stock_df = df.groupby('rcid').apply(
        get_first_non_null,
        include_groups=False
    ).reset_index()
    init_stock_df.columns = ['rcid', init_stock_var]
    df = pd.merge(df, init_stock_df, on='rcid', how='left')

df.to_parquet("output.parquet", index=False)

##### Calculate stock using estimated ρ and k parameters

In [ ]:
import pandas as pd 
import numpy as np
from statsmodels.tsa.statespace import sarimax
from tqdm import tqdm

df = pd.read_parquet("imput.parquet")
df2 = pd.read_excel("imput.xlsx")

# Parameter mapping and column definition
suffix = "_ρk"  
model_param_pairs = [
    (('AI_workforce_share_it_2', 'AI_post_share_i,t-1_2', 'AIC_init_stock_2'), 0, 'AI_2', f'AIC_stock_it_2{suffix}'),
    (('PCI_workforce_share_it_2', 'PCI_post_share_i,t-1_2', 'PCIC_init_stock_2'), 1, 'PCI_2', f'PCIC_stock_it_2{suffix}'),
    (('AIPCI_workforce_share_it_2', 'AIPCI_post_share_i,t-1_2', 'AIPCI_init_stock_2'), 2, 'AIPCI_2', f'AIPCI_stock_it_2{suffix}'),
    (('AI_workforce_share_it_5', 'AI_post_share_i,t-1_5', 'AIC_init_stock_5'), 3, 'AI_5', f'AIC_stock_it_5{suffix}'),
    (('PCI_workforce_share_it_5', 'PCI_post_share_i,t-1_5', 'PCIC_init_stock_5'), 4, 'PCI_5', f'PCIC_stock_it_5{suffix}'),
    (('AIPCI_workforce_share_it_5', 'AIPCI_post_share_i,t-1_5', 'AIPCI_init_stock_5'), 5, 'AIPCI_5', f'AIPCI_stock_it_5{suffix}'),
]

# Print estimated parameters
param_print_format = [
    (0.885579, -1.506996e-06),
    (0.880119, 8.620770e-06),
    (0.851734, -1.284692e-06),
    (0.864334, -7.438977e-08),
    (0.879546, -7.403128e-06),
    (0.862461, 9.244535e-07)
]
for i, (_, df2_idx, model_name, new_col) in enumerate(model_param_pairs):
    rho_print, k_print = param_print_format[i]
    print(f"✓ Model {i+1}/6 {model_name} - Independent parameters: ρ={rho_print:.6f}, k={k_print:.6e} → New column: {new_col}")

# Validate initial stock columns
init_stock_cols = [pair[0][2] for pair in model_param_pairs]
for col in init_stock_cols:
    if col not in df.columns:
        raise ValueError(f"Initial stock column {col} does not exist")
print(f"\nInitial dataset shape: ({df.shape[0]}, {df.shape[1]}) (contains {df.shape[1]} columns)")

# State space model fitting function
def fit_state_space_model(group, stock_var, posting_share_lag_var, init_stock_var, target_rho, target_kappa):
    group = group.sort_values('year_quarter').reset_index(drop=True)
    rcid = group['rcid'].iloc[0]
    init_stock = group[init_stock_var].iloc[0]
    
    if len(group) < 2:
        print(f"Note: Firm {rcid} time series length={len(group)}, fill with initial stock")
        group['temp_latent_var'] = init_stock
        return group
    
    y = group[stock_var].dropna().values
    X = group[posting_share_lag_var].dropna().values
    
    if len(y) < 2 or len(X) < 2:
        print(f"Note: Firm {rcid} insufficient valid samples, fill with initial stock")
        group['temp_latent_var'] = init_stock
        return group
    
    X = X.reshape(-1, 1)
    
    try:
        base_model = sarimax.SARIMAX(
            endog=y,
            exog=X,
            order=(1, 0, 0),
            trend='c',
            enforce_stationarity=True,
            enforce_invertibility=False,
            missing='drop'
        )
        
        params = base_model.start_params
        param_names = base_model.param_names
        
        for i, name in enumerate(param_names):
            if 'ar.L1' in name:
                params[i] = target_rho
            elif 'x1' in name:
                params[i] = target_kappa
            elif 'intercept' in name:
                params[i] = 0.0
        
        results = base_model.smooth(params, transformed=False)
        
        latent_states = results.filtered_state[0]
        latent_series = pd.Series(index=group.index, dtype=float)
        valid_idx = group[stock_var].notna().index
        latent_series.loc[valid_idx[:len(latent_states)]] = latent_states
        latent_series = latent_series.fillna(init_stock)
        
        group['temp_latent_var'] = latent_series
        
    except Exception as e:
        error_msg = str(e)[:50]
        print(f"Warning: Firm {rcid} fitting failed, fill with initial stock, error: {error_msg}")
        group['temp_latent_var'] = init_stock
    
    return group

# Batch fit state space models (keep all firms)
final_df = df.copy(deep=True)
unique_rcids = final_df['rcid'].unique()
print(f"\nTotal number of firms for fitting: {len(unique_rcids)} (keep all firms)")

for idx, ((stock_var, posting_lag_var, init_stock_var), df2_idx, model_name, new_col) in enumerate(model_param_pairs):
    print(f"\n=== Fitting Model {idx+1}/6: Generate new column {new_col} ({model_name}) ===")
    target_rho = df2.iloc[df2_idx]['rho_hat']
    target_kappa = df2.iloc[df2_idx]['kappa_hat']

    current_results = []
    for rcid in tqdm(unique_rcids, desc=f"Model {idx+1} progress", position=0, leave=True):
        group = final_df[final_df['rcid'] == rcid].copy()
        processed_group = fit_state_space_model(
            group=group,
            stock_var=stock_var,
            posting_share_lag_var=posting_lag_var,
            init_stock_var=init_stock_var,
            target_rho=target_rho,
            target_kappa=target_kappa
        )
        current_results.append(processed_group)

    # Merge results (keep all rows)
    temp_df = pd.concat(current_results, ignore_index=True)[['rcid', 'year_quarter', 'temp_latent_var']]
    final_df = final_df.merge(temp_df, on=['rcid', 'year_quarter'], how='left', suffixes=('', '_drop'))
    # Assign new column, fill missing values with initial stock
    final_df[new_col] = final_df['temp_latent_var'].fillna(final_df[init_stock_var])
    # Clean temporary columns
    final_df = final_df.drop(columns=[col for col in final_df.columns if col.endswith('_drop') or col == 'temp_latent_var'])
    final_df = final_df.drop_duplicates(subset=['rcid', 'year_quarter']).reset_index(drop=True)

# Save and validate results
save_path = "output.parquet"
final_df.to_parquet(save_path, index=False)

print(f"All models fitted successfully (all firms retained)!")
print(f"Final dataset shape: {final_df.shape} (original {df.shape[1]} columns + 6 new columns = {len(final_df.columns)} columns)")
print(f"Complete dataset saved to: {save_path}")
print(f"Stock columns added:")
for _, _, _, new_col in model_param_pairs:
    print(f"   - {new_col}")

# Validate new column value distribution
print("\n📈 New column value distribution validation:")
for _, _, _, new_col in model_param_pairs:
    zero_count = (final_df[new_col] == 0).sum()
    non_zero_count = len(final_df) - zero_count
    unique_rcid = final_df[final_df[new_col].notna()]['rcid'].nunique()
    print(f"{new_col}:")
    print(f"  - Zero values: {zero_count} ({zero_count/len(final_df)*100:.2f}%)")
    print(f"  - Non-zero values: {non_zero_count} ({non_zero_count/len(final_df)*100:.2f}%)")
    print(f"  - Covered firms: {unique_rcid}/{len(unique_rcids)}")
    print("-" * 50)

##### Keep top 50% firms by AIC_stock

In [ ]:
import pandas as pd

df = pd.read_parquet('import.parquet')

# Sort by AIC_stock_it_2 (descending)
df_sorted = df.sort_values(by='AIC_stock_it_2_ρk', ascending=False)

# Get top 50% firm rcids
half_length = int(len(df_sorted['rcid'].unique()) * 0.5)
top_50_percent_rcid = df_sorted['rcid'].unique()[:half_length]

# Filter data (keep 44 quarters per firm)
df_filtered = df[df['rcid'].isin(top_50_percent_rcid)]

# Validate filtered data
filtered_unique_enterprises = df_filtered['rcid'].nunique()
filtered_data_length = df_filtered.shape[0]
print(f"Number of firms after filtering: {filtered_unique_enterprises}")
print(f"Number of rows after filtering: {filtered_data_length}")

# Save filtered dataset
df_filtered.to_parquet('output.parquet', index=False)

##### Calculate KPSS patent variables

In [ ]:
import pandas as pd

df1 = pd.read_parquet('import.parquet')

# Remove duplicate patents
df1_unique = df1.drop_duplicates(subset=['patent_num'])

# Define aggregation rules
agg_dict = {
    'xi_real': 'sum',
    'cites': 'sum',
    'patent_num': 'nunique'
}

# Filter AI patent data (is_aicpc=1)
df1_ai = df1_unique[df1_unique['is_aicpc'] == 1]

# Aggregate by filing quarter (_f suffix)
agg_f = df1_unique.groupby(['rcid', 'issue_year_quarter']).agg(agg_dict).reset_index()
agg_f.columns = ['rcid', 'year_quarter_f', 'real_f', 'cites_f', 'patent_count_f']

ai_agg_f = df1_ai.groupby(['rcid', 'issue_year_quarter']).agg(agg_dict).reset_index()
ai_agg_f.columns = ['rcid', 'year_quarter_f', 'ai_real_f', 'ai_cites_f', 'ai_patent_count_f']

agg_f_merged = pd.merge(agg_f, ai_agg_f, on=['rcid', 'year_quarter_f'], how='left').fillna(0)

# Aggregate by issue quarter (_i suffix)
agg_i = df1_unique.groupby(['rcid', 'filing_year_quarter']).agg(agg_dict).reset_index()
agg_i.columns = ['rcid', 'year_quarter_i', 'real_i', 'cites_i', 'patent_count_i']

ai_agg_i = df1_ai.groupby(['rcid', 'filing_year_quarter']).agg(agg_dict).reset_index()
ai_agg_i.columns = ['rcid', 'year_quarter_i', 'ai_real_i', 'ai_cites_i', 'ai_patent_count_i']

agg_i_merged = pd.merge(agg_i, ai_agg_i, on=['rcid', 'year_quarter_i'], how='left').fillna(0)

# Create complete quarterly panel (2013-01 to 2023-10)
quarters = [
    f'{y}-01' for y in range(2013, 2024)
] + [
    f'{y}-04' for y in range(2013, 2024)
] + [
    f'{y}-07' for y in range(2013, 2024)
] + [
    f'{y}-10' for y in range(2013, 2024)
]
target_quarters = [q for q in sorted(list(set(quarters))) if q >= '2013-01' and q <= '2023-10']
target_quarters = sorted(target_quarters)

unique_rcid = df1['rcid'].unique()
rcid_quarter = pd.MultiIndex.from_product([unique_rcid, target_quarters], names=['rcid', 'year_quarter']).to_frame(index=False)

# Merge aggregated results to complete panel
df1 = pd.merge(rcid_quarter, agg_f_merged, left_on=['rcid', 'year_quarter'], right_on=['rcid', 'year_quarter_f'], how='left')
df1 = pd.merge(df1, agg_i_merged, left_on=['rcid', 'year_quarter'], right_on=['rcid', 'year_quarter_i'], how='left')

# Clean columns and fill missing values with 0
df1 = df1.drop(columns=['year_quarter_f', 'year_quarter_i'], errors='ignore')
all_target_cols = [
    'real_f', 'cites_f', 'patent_count_f',
    'real_i', 'cites_i', 'patent_count_i',
    'ai_real_f', 'ai_cites_f', 'ai_patent_count_f',
    'ai_real_i', 'ai_cites_i', 'ai_patent_count_i'
]
for col in all_target_cols:
    df1[col] = df1[col].fillna(0)

df1 = df1.reset_index(drop=True)
df1.to_parquet('output.parquet', index=False)

##### Calculate Compustat firm financial variables

In [ ]:
import pandas as pd

df1 = pd.read_csv('import.csv')

# Calculate R&D intensity (xrdintensity = xrdq / saleq)
df1['xrdintensity'] = df1['xrdq'] / df1['saleq']
df1.loc[(df1['saleq'] == 0) | (df1['saleq'].isna()), 'xrdintensity'] = pd.NA
df1.loc[(df1['xrdq'].isna()) & (df1['saleq'] != 0) & (~df1['saleq'].isna()), 'xrdintensity'] = 0

# Calculate ROA (roa = niq / atq)
df1['roa'] = df1['niq'] / df1['atq']

# Calculate Tobin's Q
df1['tobinQ'] = (df1['atq'] + (df1['cshoq'] * df1['prccq']) - df1['ceqq']) / df1['atq']

# Calculate cash holding ratio (cash_at = cheq / atq)
df1['cash_at'] = df1['cheq'] / df1['atq']
df1.loc[(df1['atq'] == 0) | (df1['atq'].isna()), 'cash_at'] = pd.NA

# Calculate leverage (leverage = (dlttq + dlcq) / atq)
df1['dlttq_filled'] = df1['dlttq'].fillna(0)
df1['dlcq_filled'] = df1['dlcq'].fillna(0)
df1['leverage'] = (df1['dlttq_filled'] + df1['dlcq_filled']) / df1['atq']
df1.loc[(df1['atq'] == 0) | (df1['atq'].isna()), 'leverage'] = pd.NA
df1.drop(columns=['dlttq_filled', 'dlcq_filled'], inplace=True)

# Calculate firm age (age = current_year - ipo_year)
df1['current_year'] = df1['year_quarter'].str.split('-').str[0].astype(int)
df1['ipo_year'] = pd.NA
df1['ipodate_str'] = df1['ipodate'].astype(str)
ipo_year_format1 = df1['ipodate_str'].str.split('-').str[0]
ipo_year_format2 = df1['ipodate_str'].str[:4]
df1['ipo_year'] = pd.to_numeric(ipo_year_format1.where(ipo_year_format1.str.len() == 4, ipo_year_format2), errors='coerce')
df1.drop(columns=['ipodate_str'], inplace=True)
df1['age'] = df1['current_year'] - df1['ipo_year']
df1.loc[(df1['current_year'].isna()) | (df1['ipo_year'].isna()), 'age'] = pd.NA

df1.to_parquet('output.parquet', index=False)

##### Calculate grouping variables for heterogeneity analysis

In [ ]:
import pandas as pd
import numpy as np

df1 = pd.read_csv('import.csv')

# Keep core columns
core_columns = [
    'rcid', 'year_quarter',
    'xsgaq', 'saleq',
    'intanq', 'atq'
]
df1 = df1[[col for col in core_columns if col in df1.columns]]

# Calculate SG&A expense intensity (H1)
df1['xsgaq'] = df1['xsgaq'].fillna(0)
df1['saleq'] = df1['saleq'].fillna(0)
df1['sgna_intensity'] = np.where(
    df1['saleq'] > 0,
    df1['xsgaq'] / df1['saleq'],
    0
)

# Calculate intangible asset intensity (H2)
df1['intanq'] = df1['intanq'].fillna(0)
df1['atq'] = df1['atq'].fillna(0)
df1['intan_intensity'] = np.where(
    df1['atq'] > 0,
    df1['intanq'] / df1['atq'],
    0
)

final_columns = [
    'rcid', 'year_quarter',
    'sgna_intensity', 'intan_intensity'
]
df1 = df1[final_columns]

df1.to_csv('output.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np

df1 = pd.read_csv('import1.csv')
df2 = pd.read_csv('import2.csv')

# Merge datasets
df1 = pd.merge(df1, df2, on=['rcid', 'year_quarter'], how='left')

# Fill missing values with 0
df1['sgna_intensity'] = df1['sgna_intensity'].fillna(0)
df1['intan_intensity'] = df1['intan_intensity'].fillna(0)

# Calculate mean by rcid
sgna_mean = df1.groupby('rcid')['sgna_intensity'].mean().reset_index()
sgna_mean.rename(columns={'sgna_intensity': 'sgna_intensity_mean'}, inplace=True)

intan_mean = df1.groupby('rcid')['intan_intensity'].mean().reset_index()
intan_mean.rename(columns={'intan_intensity': 'intan_intensity_mean'}, inplace=True)

# Merge mean columns back to df1
df1 = pd.merge(df1, sgna_mean, on='rcid', how='left')
df1 = pd.merge(df1, intan_mean, on='rcid', how='left')

# Initialize binary label columns
df1['sgna_intensity_binary'] = np.nan
df1['intan_intensity_binary'] = np.nan

# Filter valid rcids (non-null patent_count_f)
valid_mask = df1['patent_count_f'].notnull()
valid_rcids = df1.loc[valid_mask, 'rcid'].unique()

# Calculate median of mean values for valid rcids
mean_data = df1.loc[df1['rcid'].isin(valid_rcids), ['rcid', 'sgna_intensity_mean', 'intan_intensity_mean']].drop_duplicates(subset='rcid')
sgna_median = mean_data['sgna_intensity_mean'].median() if not mean_data.empty else 0
intan_median = mean_data['intan_intensity_mean'].median() if not mean_data.empty else 0

print(f"Median of sgna_intensity_mean for valid rcids: {sgna_median}")
print(f"Median of intan_intensity_mean for valid rcids: {intan_median}")

# Binary labeling function (>= median = 1, < median = 0)
def binary_by_median(x, median):
    return 1 if x >= median else 0

# Assign binary labels to valid rcids
for rcid in valid_rcids:
    try:
        sgna_mean_val = mean_data.loc[mean_data['rcid'] == rcid, 'sgna_intensity_mean'].values[0]
        intan_mean_val = mean_data.loc[mean_data['rcid'] == rcid, 'intan_intensity_mean'].values[0]
    except IndexError:
        sgna_mean_val = 0
        intan_mean_val = 0
    
    sgna_label = binary_by_median(sgna_mean_val, sgna_median)
    intan_label = binary_by_median(intan_mean_val, intan_median)
    
    df1.loc[df1['rcid'] == rcid, 'sgna_intensity_binary'] = sgna_label
    df1.loc[df1['rcid'] == rcid, 'intan_intensity_binary'] = intan_label

# Count unique rcids by binary labels
valid_rcid_labels = df1.loc[df1['rcid'].isin(valid_rcids), ['rcid', 'sgna_intensity_binary', 'intan_intensity_binary']].drop_duplicates(subset='rcid')
sgna_count = valid_rcid_labels['sgna_intensity_binary'].value_counts(dropna=False)
intan_count = valid_rcid_labels['intan_intensity_binary'].value_counts(dropna=False)

print("\n=== Number of unique rcids by sgna_intensity_binary (0/1) ===")
print(f"0 (below median): {sgna_count.get(0.0, 0)}")
print(f"1 (above or equal to median): {sgna_count.get(1.0, 0)}")
print(f"Unlabeled (invalid rcids): {sgna_count.get(np.nan, 0)} (Note: rcids with missing patent_count_f)")

print("\n=== Number of unique rcids by intan_intensity_binary (0/1) ===")
print(f"0 (below median): {intan_count.get(0.0, 0)}")
print(f"1 (above or equal to median): {intan_count.get(1.0, 0)}")
print(f"Unlabeled (invalid rcids): {intan_count.get(np.nan, 0)} (Note: rcids with missing patent_count_f)")

df1.to_csv('output.csv', index=False)